# Model creation stage III (4x8)

In [1]:
%load_ext autoreload
%autoreload 2

# Setup

In [2]:
import time
import time
import socket
import serial
from tools.mlx import *
import time
import numpy as np
import json
import cv2
import multiprocessing

BAUD = 115200 # Baud rate
COM = '/dev/ttyACM2' # Serial port
ENABLE_WS = True # Enable WebSocket server. Disable if not using websocket. Script will crash otherwise.
NR_OF_SENSORS = 32 # Number of sensors

In [3]:
ser = serial.Serial(COM, BAUD, timeout=1)
ser.close()

In [4]:
udp_server_address = ('localhost', 9870)
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [5]:
import numpy as np
import cv2

def plot_colormap(queue, scale=4):
    
    is_first = True
    zero_bias = np.zeros((3, 8, 4))
    
    while True:
        all_data = queue.get()
        if all_data is None:
            break

        components = ['X', 'Y', 'Z']
        data_matrices = []
        for i in range(3):
            data = all_data[i * 32 : (i + 1) * 32]
            data_matrix = np.reshape(data, (8, 4))
            
            if is_first:
                zero_bias[i] = data_matrix
            
            data_matrix = data_matrix - zero_bias[i]
            
            data_matrices.append(data_matrix)
            
            

            # Scale the values to 0-255 for better visualization
            if i == 2:
                data_matrix = ((data_matrix + 30) / 40 * 255).astype(np.uint8)
            else:
                data_matrix = ((data_matrix + 10) / 20 * 255).astype(np.uint8)

            # Create a colormap image using OpenCV
            colormap = cv2.applyColorMap(data_matrix, cv2.COLORMAP_VIRIDIS)
            scaled_colormap = cv2.resize(colormap, (400, 800), interpolation=cv2.INTER_NEAREST)  # Adjust the size as needed

            # Display the resized colormap image
            cv2.imshow('Map ' + components[i], scaled_colormap)
            
        is_first = False
        
        # Create a blank image for the vector field
        vector_field_img = np.zeros((800, 400, 3), dtype=np.uint8)
        
        # Calculate the scaling factors
        scale_factor_x = vector_field_img.shape[1] / 4
        scale_factor_y = vector_field_img.shape[0] / 8
        
        z_data_matrix = ((data_matrices[2] + 30) / 40 * 255).astype(np.uint8)
        z_colormap = cv2.applyColorMap(z_data_matrix, cv2.COLORMAP_VIRIDIS) 
        z_data_matrix_resized = cv2.resize(z_colormap, (400, 800), interpolation=cv2.INTER_NEAREST)
       

        # Draw the vector field (quiver plot)
        for y in range(8):
            for x in range(4):
                start_point = (int(x * scale_factor_x + scale_factor_x / 2), int(y * scale_factor_y + scale_factor_y / 2))
                end_point = (
                    int(start_point[0] + data_matrices[1][y, x] * scale),  # X-component
                    int(start_point[1] - data_matrices[0][y, x] * scale)   # Y-component
                )
                cv2.arrowedLine(z_data_matrix_resized, start_point, end_point, (255, 0, 0), 2, tipLength=0.3)

        # Display the vector field image
        cv2.imshow('Vector Field', z_data_matrix_resized)

        cv2.waitKey(1)


In [6]:
def readout_for_t_seconds(t):
    
    queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=plot_colormap, args=(queue,))
    process.start()        
    
    t0 = time.time()
    
    ser.open()
    while time.time() - t0 < t:    
        
        data = ser.read(1)
        if data == b'\xAA':
            
            data_bytes = ser.read(6 * NR_OF_SENSORS)
            
            row = {}

            for i in range(NR_OF_SENSORS):
                
                # Encoding
                x = (data_bytes[i * 3 * 2] << 8) + (data_bytes[i * 3 * 2 + 1])
                y = (data_bytes[i * 3 * 2 + 2] << 8) + (data_bytes[i * 3 * 2 + 3])
                z = (data_bytes[i * 3 * 2 + 4] << 8) + (data_bytes[i * 3 * 2 + 5])

                x = x / 2**16 * 20 + (-10)
                y = y / 2**16 * 20 + (-10)
                z = z / 2**16 * 40 + (-30)
                
                row[f"G_x{i}"] = x
                row[f"G_y{i}"] = y
                row[f"G_z{i}"] = z      
                
            queue.put(np.array([row[f"G_x{i}"] for i in range(NR_OF_SENSORS)] + [row[f"G_y{i}"] for i in range(NR_OF_SENSORS)] + [row[f"G_z{i}"] for i in range(NR_OF_SENSORS)]))    
            json_data = json.dumps(row)
            udp_socket.sendto(json_data.encode(), udp_server_address)
    
    queue.put(None)
    ser.close()

In [8]:
ser.close()
readout_for_t_seconds(300)

KeyboardInterrupt: 

In [9]:
ser.close()